In [96]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ISLP.models import ModelSpec as MS
import statsmodels.api as sm
import keras
from keras import layers
from keras import ops
from sklearn.model_selection import train_test_split

In [97]:
Df = pd.read_csv('Faellesdata_cleaned.csv')

x = Df.drop('IsMigratorInt', axis=1) 
for i in range(0, x.shape[1]): #Standardizing the data
    x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()

x_train, x_test, y_train, y_test = train_test_split(x, Df['IsMigratorInt'], test_size=0.2, random_state=42)

In [98]:
x_train = MS(x_train, intercept = False).fit_transform(x_train)

In [99]:
x_train.shape

(7999, 15)

In [100]:
modelnn = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [101]:
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=10)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7881 - loss: 0.4600
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8394 - loss: 0.3498
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 953us/step - accuracy: 0.8451 - loss: 0.3144
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8613 - loss: 0.2926  
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.8629 - loss: 0.2880
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - accuracy: 0.8644 - loss: 0.2815
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 940us/step - accuracy: 0.8650 - loss: 0.2728
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - accuracy: 0.8688 - loss: 0.2683
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - accuracy: 0.8727 - loss: 0.2632
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - accuracy: 0.8779 - loss: 0.2633


In [102]:
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8658 - loss: 0.2652  


[0.26908978819847107, 0.8650000095367432]

In [103]:
from sklearn.metrics import accuracy_score

In [104]:
print(accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
0.865


In [105]:
modelsoftmax = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [106]:
modelsoftmax.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelsoftmax.fit(x_train, keras.utils.to_categorical(y_train), epochs=10)


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 969us/step - accuracy: 0.7920 - loss: 0.4492
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 902us/step - accuracy: 0.8457 - loss: 0.3290
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - accuracy: 0.8530 - loss: 0.3069
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8486 - loss: 0.3039  
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - accuracy: 0.8623 - loss: 0.2758
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - accuracy: 0.8714 - loss: 0.2699
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8724 - loss: 0.2685  
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.8792 - loss: 0.2603
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8802 - loss: 0.2572
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8789 - loss: 0.2574


In [107]:
modelsoftmax.evaluate(MS(x_test, intercept = False).fit_transform(x_test), keras.utils.to_categorical(y_test))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8610 - loss: 0.2652  


[0.2718632221221924, 0.859000027179718]

In [108]:
print(accuracy_score(y_test, np.argmax(modelsoftmax.predict(MS(x_test, intercept = False).fit_transform(x_test)), axis=1)))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
0.859


In [119]:
modelnn = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(1, activation='sigmoid')
])
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100, verbose = 0)
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8682 - loss: 0.2622  


[0.2717018127441406, 0.8629999756813049]

In [116]:
modelnn = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dropout(0.5),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [118]:
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100, verbose = 0)
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8530 - loss: 0.2961  


[0.29632413387298584, 0.8495000004768372]